<a href="https://colab.research.google.com/github/Vijayaranagn/Projects/blob/main/Stock_Market_Performance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

Now below is how we can collect real-time stock market data using the yfinance API:

In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime

In [7]:
start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

In [8]:
tickers = ['AAPL', "MSFT", "NFLX", 'GOOG']
df_list = []

for ticker in tickers:
  data = yf.download(ticker, start = start_date, end=end_date)
  df_list.append(data)

df = pd.concat(df_list, keys = tickers, names = ["Ticker", "Date"])
df.head(5)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-03-06  153.789993  156.300003  153.460007  153.830002  153.617523   
       2023-03-07  153.699997  154.029999  151.130005  151.600006  151.390610   
       2023-03-08  152.809998  153.470001  151.830002  152.869995  152.658844   
       2023-03-09  153.559998  154.539993  150.229996  150.589996  150.381989   
       2023-03-10  150.210007  150.940002  147.610001  148.500000  148.294876   

                     Volume  
Ticker Date                  
AAPL   2023-03-06  87558000  
       2023-03-07  56182000  
       2023-03-08  47204800  
       2023-03-09  53833600  
       2023-03-10  68572400

In this dataset, the Date column is the index column in the DataFrame. We need to reset the index before moving forward:

In [10]:
df = df.reset_index()
df.head(5)

,index,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,0,AAPL,2023-03-06,153.789993,156.300003,153.460007,153.830002,153.617523,87558000
1,1,AAPL,2023-03-07,153.699997,154.029999,151.130005,151.600006,151.390610,56182000
2,2,AAPL,2023-03-08,152.809998,153.470001,151.830002,152.869995,152.658844,47204800
3,3,AAPL,2023-03-09,153.559998,154.539993,150.229996,150.589996,150.381989,53833600
4,4,AAPL,2023-03-10,150.210007,150.940002,147.610001,148.500000,148.294876,68572400


Now let’s have a look at the performance in the stock market of all the companies:

In [11]:
import plotly.express as px

In [16]:
fig = px.line(df, x = 'Date', y="Close", color = 'Ticker', title = "Stock Market Performance for Last 3 Months")
fig.show()

Now let’s look at the faceted area chart,

In [19]:
fig = px.area(df, x = "Date", y="Close", color = 'Ticker', facet_col = 'Ticker',
              labels = {"Date":'Date', "Close": "Closing Price", "Ticker":"Company"}, 
              title = "Stock Market price for Apple, Microsoft, Netflix, and Google")
fig.show()

Now let’s analyze moving averages, which provide a useful way to identify trends and patterns in each company’s stock price movements over a period of time:

In [20]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
59  173.829001  172.543001
60  174.347000  172.978501
61  175.087000  173.610501
62  175.676999  174.368501
63  176.118999  174.669001

[64 rows x 2 columns]
Moving Averages for GOOG
           MA10       MA20
192         NaN        NaN
193         NaN        NaN
194         NaN        NaN
195         NaN        NaN
196         NaN        NaN
..          ...        ...
251  123.356000  116.86625
252  123.684000  117.73575
253  123.973000  118.64825
254  124.144001  119.64925
255  124.482001  120.67000

[64 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
..          ...         ...
123  320.291000  314.263998
124  32

Now here’s how to visualize the moving averages of all companies:



In [22]:
for ticker, group in df.groupby("Ticker"):
  fig = px.line(group, x = "Date", y =['Close', 'MA10', 'MA20'], title = f"{ticker} Moving Averages")
  fig.show()

The output shows four separate graphs for each company. When the MA10 crosses above the MA20, it is considered a bullish signal indicating that the stock price will continue to rise. Conversely, when the MA10 crosses below the MA20, it is a bearish signal that the stock price will continue falling.

Let us now analyze the volatility of all companies. Volatility is a measure of how much and how often the stock price or market fluctuates over a given period of time. Here’s how to visualize the volatility of all companies:

In [26]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
fig.show()

High volatility indicates that the stock or market experiences large and frequent price movements, while low volatility indicates that the market experiences smaller or less frequent price movements.

Now let’s analyze the correlation between the stock prices of Apple and Microsoft:



In [27]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Apple and Microsoft')
fig.show()

There is a strong linear relationship between the stock prices of Apple and Microsoft, which means that when the stock price of Apple increases, the stock price of Microsoft also tends to increase. It is a sign of a strong correlation or similarity between the two companies, which can be due to factors such as industry trends, market conditions, or common business partners or customers. For investors, this positive correlation may indicate an opportunity to diversify their portfolio by investing in both companies, as both stocks may offer similar potential returns and risks.